In [8]:
%load_ext autoreload
%autoreload 2

#standard imports
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import sys
sys.path.append("..")

#rl book imports
import rl
from rl.markov_decision_process import MarkovDecisionProcess
from rl.markov_process import State, MarkovProcess, NonTerminal, Terminal

from typing import (Callable, Dict, Generic, Iterator, Iterable, List,
                    Mapping, Optional, Sequence, Tuple, TypeVar, overload)

from rl.distribution import Categorical, Distribution, Constant, Choose
from rl.policy import Policy
from rl.monte_carlo import epsilon_greedy_policy, greedy_policy_from_qvf, glie_mc_control, mc_prediction
from rl.function_approx import LinearFunctionApprox, AdamGradient
from rl.td import glie_sarsa, q_learning


#custom imports 
import utils as u
import data as dat
import mdp_agent as ag
import baseline_policies as bp
import q_plots as qp
import backtest as btest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#  Simulate data

In [37]:
mu=100
sigma = 30
train, test = dat.build_simulated_train_test(N=10,mu=mu,sigma=sigma)
u.plot_plotly_multiple(train)

In [38]:
u.plot_plotly(test)

In [40]:
trader = ag.Trading(train,test)
bt = btest.Backtester(trader, bp.BuyAndHold())
bt.summary()

/Users/mehdi/Desktop/cme241/RL_Project/utils.py:96: FutureWarning:

Argument `closed` is deprecated in favor of `inclusive`.



In [42]:
mrp_buy = trader.apply_policy(bp.BuyAndHold())

In [49]:
initial_state = trader.generate_start_state("train")
traces = mrp_buy.reward_traces(initial_state)

In [ ]:
mc_prediction(
    traces=traces,
    approx_0=Tabular(),
    gamma=user_gamma,
    episode_length_tolerance=1e-6
)